In [ ]:
from keras.models import Model
from keras.layers import Input
from ae_model import encoder_vgg16, decoder1_vgg16, decoder2_vgg16, decoder3_vgg16, encoder1_vgg16

input_shape = Input(shape=[256, 192, 3])
ae = Model(input_shape, decoder1_vgg16(encoder_vgg16(input_shape)))
# ae.summary()

In [4]:
# from json import load
# import numpy as np
# from keras.preprocessing.image import img_to_array, load_img
# img_path = './grid-search/dataset-bc/JPEGImages/BloodImage_00000.jpg'

# img_data = []

# img_data.append(img_to_array(load_img(img_path, target_size=(192, 256))))

# train_array = np.reshape(img_data, (len(img_data), 256, 192, 3))
# train_array = train_array.astype('float32') / 255. 

# train_array_2d = np.reshape(train_array, (train_array.shape[0], -1))

# print(train_array.shape)
# print(train_array_2d.shape)

import os
import cv2
import numpy as np
from tqdm import tqdm
from keras.preprocessing.image import img_to_array, load_img

datasets = []
path = './grid-search/dataset-bc/JPEGImages/'

files = os.listdir(path)


for i in tqdm(files):
    img = load_img(path+i, target_size=(192, 256))
    datasets.append(img_to_array(img))

data_array = np.reshape(datasets, (len(datasets), 256, 192, 3))
data_array = data_array.astype('float32') / 255.

data_array_2d = np.reshape(data_array, (data_array.shape[0], -1)) #sklearn need 2d array

print(img.size)
print(data_array.shape)
print(data_array_2d.shape)


100%|██████████| 366/366 [00:00<00:00, 882.97it/s]

(256, 192)
(366, 256, 192, 3)
(366, 147456)


In [ ]:
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasRegressor


def create_model(m):
    
    m.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

    return m

tf.random.set_seed(7)

grid = KerasRegressor(model=create_model(ae))

grid_batch_size = [8, 16, 32]
grid_epoch = [50, 100]

param_grid = dict(batch_size = grid_batch_size, epochs = grid_epoch)

grid_cv = GridSearchCV(estimator=grid, param_grid=param_grid, verbose=2)

grid_cv.fit(data_array, data_array_2d)


In [ ]:
print("Best: %f using %s" % (grid_cv.best_score_, grid_cv.best_params_))
means = grid_cv.cv_results_['mean_test_score']
stds = grid_cv.cv_results_['std_test_score']
params = grid_cv.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))